In [1]:
import pandas as pd
from collections import Counter
import spacy
sp = spacy.load('en_core_web_sm')
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'idea', 'proposal', 'plastic'])
import gensim.downloader as api
import gensim
from gensim.utils import simple_preprocess
import json
import pytextrank
import re

In [13]:
# run only the first time
word_vectors = api.load("glove-wiki-gigaword-100")

In [3]:
df = pd.read_json('cc_sent.json')
# df = pd.read_json('env_sent.json')

df.head(5)

,idea,description,task,comment,dominant_topic,topic_perc_contrib,keywords,sentiment,sentiment_score,presentation
0,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,A structure or monument at the Nudillo could i...,2,0.1859,"impact, people, chime, village, fine, connecti...",positive,0.100000,A structure or monument at the Nudillo could i...
1,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,The built form should be lit at night.,1,0.2016,"city, build, live, road, solution, beauty, put...",positive,0.100000,The built form should be lit at night.
2,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,"More than a plaza, the space can be activated ...",4,0.2037,"structure, good, realm, car, bottom, gray, par...",positive,0.085714,"More than a plaza, the space can be activated ..."
3,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,The structure could incorporate an observation...,0,0.2163,"place, community, increase, gather, port, dens...",negative,-0.012500,It could echo the tower elements of the Califo...
4,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,I think its a great idea,0,0.1830,"place, community, increase, gather, port, dens...",positive,0.300000,I think its a great idea


In [4]:
# clean text

def clean_and_embed (data):
    data = re.sub('\S*@\S*\s?', '', data) 
    data = re.sub('\s+', ' ', data)
    data = re.sub("\'", "", data)
    text = sp(data)
    tokens = [word.text for word in text]
    tokens = [word for word in simple_preprocess(str(text)) if word not in stop_words]
    text = ' '.join(word for word in tokens)
#     print(text)
    return text

In [5]:
# find number of comments that contain the topic keyword

def get_keyword_leaf_size(k, topic_rows):    
    count = 0
        
    for c in topic_rows["comment"]:
        text = (clean_and_embed(c))
                
        if k in text:
            count += 1
        else:
            if k in word_vectors.wv.vocab:
                similar_keys = word_vectors.most_similar(positive=[k]) 
                for i in similar_keys:
                    if i[0] in text:
                        count += 1
    return count

In [6]:
# find keyphrase to represent topic sentences

def get_keyphrase(prop_rows, t):
    text = ""
    rows = prop_rows.loc[prop_rows['dominant_topic'] == t]
    for c in rows["comment"]:
        text = text + " " + c
        
    nlp = spacy.load("en_core_web_sm")
    tr = pytextrank.TextRank(logger=None)
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
    doc = nlp(text)
    
    if (doc._.phrases):
        return doc._.phrases[0].text
    else:
        return rows.iloc[0]["keywords"].split(", ")[0]

In [7]:
# prop_rows = df.loc[df['idea'] == "Public Art"]
# topics = Counter(prop_rows["dominant_topic"])

# topic_rows = df.loc[(df['idea'] == "Public Art") & (df['dominant_topic'] == t)]

# keywords = (topic_rows["keywords"].iloc[0]).split(", ")
# for k in keywords:
#     get_keyword_leaf_size(k, topic_rows)

In [8]:
# root

cc_obj = {}
cc_obj["name"] = "Proposals"
cc_obj["children"] = []

In [9]:
# rows["keywords"][0].split(", ")[0]

In [10]:
# proposals

proposals = Counter(df["idea"])

for p in proposals:
    child_prop = {}
    child_prop["name"] = p
    child_prop["children"] = []
    cc_obj["children"].append(child_prop)

In [11]:
# topics

for p in proposals:
    prop_rows = df.loc[df['idea'] == p]
    topics = Counter(prop_rows["dominant_topic"])
    for t in topics:
        for i in range (0, len(cc_obj["children"])):
            if cc_obj["children"][i]["name"] == p:
                child_top = {}
                child_top["dom_top"] = t
                child_top["name"] = get_keyphrase(prop_rows, t)
                child_top["children"] = []
                cc_obj["children"][i]["children"].append(child_top)

In [14]:
# keywords

for p in proposals:
    prop_rows = df.loc[df['idea'] == p]
    topics = Counter(prop_rows["dominant_topic"])
    for t in topics:
        topic_rows = df.loc[(df['idea'] == p) & (df['dominant_topic'] == t)]
        keywords = (topic_rows["keywords"].iloc[0]).split(", ")
        for k in keywords:
            if get_keyword_leaf_size(k, topic_rows) > 0:
                for i in range (0, len(cc_obj["children"])):
                    if cc_obj["children"][i]["name"] == p:
                        for j in range(0, len(cc_obj["children"][i]["children"])):
                            if cc_obj["children"][i]["children"][j]["dom_top"] == t:
                                child_keyword = {}
                                child_keyword["name"] = k
                                child_keyword["size"] = get_keyword_leaf_size(k, topic_rows)
                                cc_obj["children"][i]["children"][j]["children"].append(child_keyword)

/home/mjasim/environments/env3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


In [16]:
# cc_obj

In [132]:
# with open('cc_viz.json', 'w') as outfile:
#     json.dump(cc_obj, outfile)

# with open('env_viz.json', 'w') as outfile:
#     json.dump(cc_obj, outfile)

In [53]:
# for forced directed graph

fd_obj_list = []
proposals = Counter(df["idea"])
for p in proposals:
    prop_obj = {}
    prop_nodes = []
    prop_links = []
    prop_rows = df.loc[df['idea'] == p]
    topics = set(prop_rows["dominant_topic"])
    for t in topics:
        for idx, row in prop_rows.iterrows():
            if(row['dominant_topic' == t]):
                kws = row['keywords'].split(', ')
                break;
    
        topic_rt = get_keyphrase(prop_rows, t)
        prop_nodes.append({"id": topic_rt + "_" + str(t), "group": t, "label": topic_rt})
        for k in kws:
            prop_nodes.append({"id": k + "_" + str(t), "group": t, "label": k})
            prop_links.append({"source": k + "_" + str(t), "target": topic_rt + "_" + str(t), "value": 1})
    
    prop_obj = {"nodes": prop_nodes, "links": prop_links}
    
    fd_obj_list.append(prop_obj)
    
#     print(topics)
# for t in topics:
#     for i in range (0, len(cc_obj["children"])):
#         if cc_obj["children"][i]["name"] == p:
#             child_top = {}
#             child_top["dom_top"] = t
#             child_top["name"] = get_keyphrase(prop_rows, t)
#             child_top["children"] = []
#             cc_obj["children"][i]["children"].append(child_top)
                

In [54]:
with open('cc_fd_graph.json', 'w') as outfile:
    json.dump(fd_obj_list, outfile)

In [55]:
proposals

Counter({'Build a tower': 36,
         'Public Art': 23,
         'Roundabout': 26,
         'Huge Fountain': 26,
         'Interactive Art': 28,
         'EL Parquecito': 16,
         'Pedestrian Destination': 20,
         'Promenade Real': 25,
         'Platform connected with bridges': 34,
         'Triangle platform with 3 pillars and flat bridge': 1,
         'Zipline': 21,
         'Transit Hub': 21,
         'Gateway': 7,
         'Connecting the past to the future': 19,
         'Cart before the Horse': 6,
         'Call the Area Something Else.': 3,
         'The Diamond': 4,
         'The Lyre': 5,
         'EV / Barrio Market': 23})